Airline Delay Analysis


Metadata:
YEAR: Year of the Flight Trip
MONTH: Month of the Flight Trip
DAY: Day of the Flight Trip
DAY_OF_WEEK: Day of Week of the Flight Trip
AIRLINE: Airline Identifier
FLIGHT_NUMBER: Flight Identifier
TAIL_NUMBER: Aircraft Identifier
ORIGIN_AIRPORT: Starting Airport
DESTINATION_AIRPORT: Destination Airport
SCHEDULED_DEPARTURE: Planned Departure Time
DEPARTURE_TIME: WHEEL_OFF - TAXI_OUT
DEPARTURE_DELAY: Total Delay on Departure
TAXI_OUT: The time duration elapsed between departure from the origin airport gate and wheels off
WHEELS_OFF: The time point that the aircraft's wheels leave the ground
SCHEDULED_TIME: Planned time amount needed for the flight trip
ELAPSED_TIME: AIR_TIME + TAXI_IN + TAXI_OUT
AIR_TIME: The time duration between WHEELS_OFF and WHEELS_ON time
DISTANCE: Distance between 2 airports
WHEELS_ON: The time point that the aircraft's wheels touch on the ground
TAXI_IN: The time duration elapsed between WHEELS_ON and gate arrival at the destination airport
SCHEDULED_ARRIVAL: Planned arrival time
ARRIVAL_TIME: WHEELS_ON + TAXI_IN
ARRIVAL_DELAY: ARRIVAL_TIME - SCHEDULED_ARRIVAL
DIVERTED: Aircraft landed on airport that out of schedule
CANCELLED: Flight cancelled (1 = cancelled)
CANCELLATION_REASON: Reason of cancellation of flight: A - Airline/Carrier, B - Weather, C - National Air System, D - Security
AIR_SYSTEM_DELAY: Delay caused by air system
SECURITY_DELAY: Delay caused by security
AIRLINE_DELAY: Delay caused by the airline
LATE_AIRCRAFT_DELAY: Delay caused by the aircraft
WEATHER_DELAY: Delay cause by weather


In [1]:
pip install --upgrade imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade ydata-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.6/357.6 kB 4.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: ydata-profiling
    Found existing installation: ydata-profiling 4.6.2
    Uninstalling ydata-profiling-4.6.2:
      Successfully uninstalled ydata-profiling-4.6.2
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade sweetviz

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# Class balancing
from imblearn.over_sampling import SMOTE

# Model Selection and Evaluation
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
recall_score, accuracy_score, confusion_matrix,
classification_report, f1_score, precision_score,
precision_recall_fscore_support
)
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go


from pandas_profiling import ProfileReport
import sweetviz as sv


# Word Cloud
from wordcloud import STOPWORDS, WordCloud

# Configuration
# Remove the limit for the number of displayed columns
pd.set_option("display.max_columns", None)

# Set the limit for the number of displayed rows
pd.set_option("display.max_rows", 200)

In [6]:
# List of time columns to be converted
time_columns = ['SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'SCHEDULED_TIME', 'WHEELS_OFF', 'WHEELS_ON', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME']

In [7]:
airlines_df = pd.read_csv('airlines.csv')
airports_df = pd.read_csv('airports.csv')
flights_df = pd.read_csv('flights.csv')

In [8]:
print(airlines_df.shape)
airlines_df.head()

(14, 2)


,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [9]:
print(airports_df.shape)
airports_df.head()

(322, 7)


,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [10]:
print(flights_df.shape)
flights_df.head()

(5819079, 31)


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
airlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   IATA_CODE  14 non-null     object
 1   AIRLINE    14 non-null     object
dtypes: object(2)
memory usage: 352.0+ bytes


In [12]:
airports_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   IATA_CODE  322 non-null    object 
 1   AIRPORT    322 non-null    object 
 2   CITY       322 non-null    object 
 3   STATE      322 non-null    object 
 4   COUNTRY    322 non-null    object 
 5   LATITUDE   319 non-null    float64
 6   LONGITUDE  319 non-null    float64
dtypes: float64(2), object(5)
memory usage: 17.7+ KB


In [13]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [14]:
def categorize_delay(x):
    if x > 60:
        return 3
    elif x > 30:
        return 2
    elif x > 15:
        return 1
    else:
        return 0

flights_df['LATE_AIRCRAFT_DELAY_CAT'] = flights_df['LATE_AIRCRAFT_DELAY'].apply(lambda x: categorize_delay(x))

In [15]:
# Convert 'Date' to 'dd/mm/yyyy' format
def transform_date_columns(df, year_col, month_col, day_col):
    df['Date'] = pd.to_datetime(df[[year_col, month_col, day_col]].astype(str).agg('/'.join, axis=1), format='%Y/%m/%d').dt.strftime('%d/%m/%Y')

In [16]:
# Convert time columns to the right format (HHmm)
def transform_time_columns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: format_time(x))

def format_time(time_str):
    try:
        # Convert the time value to an integer (removing the decimal point and trailing zeros)
        time_as_int = int(float(time_str))
        # Ensure the time string has the correct format by zero-filling
        time_str = str(time_as_int).zfill(4)
        
        # Extract hours and minutes using string slicing
        hours = time_str[:2]
        minutes = time_str[2:]
        # Try to parse the time using the expected format
        # time_obj = pd.to_datetime(time_str, format='%H%M').time()
        # return time_obj.strftime('%I:%M %p')
        ampm = 'AM'
        if int(hours) > 12:
            number = int(hours) - 12
            if number < 10:
                hours = f'0{str(number)}'
            else:
                hours = f'{str(number)}'
            ampm = 'PM'

        time_str = f'{hours}:{minutes} {ampm}'
        return time_str
    except ValueError:
        # Handle unexpected values, for example, replace with NaN
        return pd.NaT

In [17]:
transform_date_columns(flights_df, 'YEAR', 'MONTH', 'DAY')

In [18]:
transform_time_columns(flights_df, time_columns)

In [19]:
# Assuming 'transformed_flights.csv' is the new file
flights_df.to_csv('transformed_flights.csv', index=False)

In [20]:
# Display the first few rows of the transformed DataFrame
flights_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,LATE_AIRCRAFT_DELAY_CAT,Date
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,00:05 AM,11:54 PM,-11.0,21.0,00:15 AM,02:05 AM,194.0,169.0,1448,04:04 AM,4.0,04:30 AM,04:08 AM,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,01/01/2015
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,00:10 AM,00:02 AM,-8.0,12.0,00:14 AM,02:80 AM,279.0,263.0,2330,07:37 AM,4.0,07:50 AM,07:41 AM,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,01/01/2015
2,2015,1,1,4,US,840,N171US,SFO,CLT,00:20 AM,00:18 AM,-2.0,16.0,00:34 AM,02:86 AM,293.0,266.0,2296,08:00 AM,11.0,08:06 AM,08:11 AM,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,01/01/2015
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,00:20 AM,00:15 AM,-5.0,15.0,00:30 AM,02:85 AM,281.0,258.0,2342,07:48 AM,8.0,08:05 AM,07:56 AM,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,01/01/2015
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,00:25 AM,00:24 AM,-1.0,11.0,00:35 AM,02:35 AM,215.0,199.0,1448,02:54 AM,5.0,03:20 AM,02:59 AM,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,01/01/2015


In [21]:
flights_df_profile = ProfileReport(flights_df, title="Profiling Report")

In [22]:
flights_df_profile.to_file("y_profiling_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
flights_df_report = sv.analyze(flights_df)
flights_df_report.show_html(filepath='sweetviz_report.html', 
            open_browser=True, 
            layout='widescreen', 
            scale=None)

                                             |          | [  0%]   00:00 -> (? left)

TypeError: 

Column [ORIGIN_AIRPORT] has a 'mixed' inferred_type (as determined by Pandas).
This is is not currently supported; column types should not contain mixed data.
e.g. only floats or strings, but not a combination.

POSSIBLE RESOLUTIONS:
BEST -> Make sure series [ORIGIN_AIRPORT] only contains a certain type of data (numerical OR string).
OR -> Convert series [ORIGIN_AIRPORT] to a string (if makes sense) so it will be picked up as CATEGORICAL or TEXT.
     One way to do this is:
     df['ORIGIN_AIRPORT'] = df['ORIGIN_AIRPORT'].astype(str)
OR -> Convert series [ORIGIN_AIRPORT] to a numerical value (if makes sense):
     One way to do this is:
     df['ORIGIN_AIRPORT'] = pd.to_numeric(df['ORIGIN_AIRPORT'], errors='coerce')
     # (errors='coerce' will transform string values to NaN, that can then be replaced if desired; consult Pandas manual pages for more details)
